In [1]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.3.1
    Uninstalling sentence-transformers-2.3.1:
      Successfully uninstalled sentence-transformers-2.3.1


In [1]:
#e5-mistral-7b-instruct,paraphrase-MiniLM-L6-v2
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Sentences we want to encode. Example:
sentence = ['This framework generates embeddings for each input sentence']

# Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)

/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
embedding[0].shape

(384,)

In [4]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [4]:
!pip install -U openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 11.9 MB/s eta 0:00:00


In [7]:
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:8080/v1", api_key="NONE")

response = client.chat.completions.create(
        model="mistral",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Who won the world series in 2020?"},
            {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
            {"role": "user", "content": "Where was it played?"}
        ]
)
print(response)

ChatCompletion(id='ae90f5db-bf93-43c3-82a4-c0a999579d3d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The 2020 World Series was played at Arlington, Texas, at the Globe Life Field and the American Airlines Center.\n', role='assistant', function_call=None, tool_calls=None))], created=1710133036, model='mistral', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=0, prompt_tokens=0, total_tokens=0))


In [10]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:8080/v1", api_key="NONE")

response = client.chat.completions.create(
        model="mistral",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Who won the world series in 2020?"},
            {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
            {"role": "user", "content": "Where was it played?"}
        ]
)
print(response)

ChatCompletion(id='7b263a9a-74cf-4ae2-817c-7407675e7c15', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The 2020 World Series was played at Arlington, Texas, at the Globe Life Field and the American Airlines Center.\n', role='assistant', function_call=None, tool_calls=None))], created=1709095644, model='mistral', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=0, prompt_tokens=0, total_tokens=0))


In [6]:
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:8080/v1", api_key="NONE")

response = client.embeddings.create(
        input="Your text string goes here",
        model="text-embedding-ada-002"
)

print(len(response.data[0].embedding))

1024


In [13]:
! pip install pypdf2 rich

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 12.6 MB/s eta 0:00:00


In [15]:
import io
import json
import uuid

import openai
from openai import OpenAI
import PyPDF2
import requests

import numpy as np

from rich.console import Console
from rich.progress import track, Progress
from pathlib import Path
from functools import cache

from prompts import DEFAULT_QUERY_PROMPT_TMPL

In [18]:
console = Console()
client = OpenAI(base_url="http://localhost:8080/v1", api_key="NONE")

In [19]:
CACHE_DIR = "./cache"
PDF_URL = "https://arxiv.org/pdf/1706.03762.pdf"
Path(CACHE_DIR).mkdir(exist_ok=True)

In [20]:
@cache
def get_embedding(text):
    response = client.embeddings.create(
		input=text,
		model="text-embedding-ada-002"
	)
    
    return response.data[0].embedding

In [8]:
def query_model(prompt):
	response = client.chat.completions.create(
		model="mistral",
		messages=[{"role": "user", "content": prompt}],
		temperature=0.7,
		max_tokens=512,
		top_p=1,
		frequency_penalty=0,
		presence_penalty=0
	)
	
	return response.choices[0].message.content

In [22]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [23]:
def query_pdf(pdf_url, query):
    pdf_filename = Path(pdf_url).name
    
    local_file = Path(CACHE_DIR) / pdf_filename
    
    # first get the pdf if not there
    if not local_file.exists():
        with console.status("[bold yellow]downloading pdf...", spinner="runner"):
            # download and save in cache dir
            pdf_res = requests.get(pdf_url, timeout=300)
            # cache the pdf
            local_file.write_bytes(pdf_res.content)

        console.log("[bold green]pdf downloaded!")
    
    # check if embeddings exists in cache
    embedding_file = local_file.parent / f"{local_file.stem}.embeddings.json"
    
    if embedding_file.exists():
        with console.status(f"[bold yellow]loading embedding index from cache...", spinner="balloon"):
            with embedding_file.open("r") as f:
                embeddings_index = json.load(f)
            console.log("[bold green]embedding index loaded from cache")
    else:
        with console.status(f"[bold yellow]computing embeddings and index...", spinner="earth"):
            with local_file.open("rb") as f:
                pdf = PyPDF2.PdfReader(f)
                # compute embeddings and save to disk
                num_pages = len(pdf.pages)
                console.log(f"got pages: {num_pages=}")
            
                embeddings_index = {
                    "sequence": [],
                    "docs_store": {},
                    "vecs_store": {}
                }
                
                for page_num in track(range(num_pages), description="querying embeddings..."):
                    page_uuid = str(uuid.uuid4())
                    pdf_text = pdf.pages[page_num].extract_text()
                    
                    embeddings_index['docs_store'][page_uuid] = {
                        "id": page_uuid,
                        "text": pdf_text,
                        "page_num": page_num
                    }
                    
                    text_embedding = get_embedding(pdf_text)

                    embeddings_index['vecs_store'][page_uuid] = {
                        "id": page_uuid,
                        "vector": text_embedding,
                    }
                    
                    embeddings_index['sequence'].append(page_uuid)
                    
                # save this index
                with embedding_file.open("w") as f:
                    json.dump(embeddings_index, f)

        console.log("[bold green]done computing embedding index")
        
    # create embeddings numpy
    np_index_to_uuid = []
    embeddings_np = []
    for emb_uuid, emb in embeddings_index['vecs_store'].items():
        np_index_to_uuid.append(emb_uuid)
        embeddings_np.append(emb['vector'])
    embeddings_np = np.array(embeddings_np)
    
    # get query embed
    query_emb = get_embedding(query)
    
    # find the closest text to the query in doc store
    with console.status(f"[bold yellow]finding relevant text in doc...", spinner="earth"):
        relevant_text = embeddings_index['docs_store'][
            np_index_to_uuid[
                np.argmax([
                    cosine_similarity(query_emb, emb) for emb in embeddings_np
                ])
            ]
        ]['text']
    console.log("[bold green]found relevant text from input")
    
    query_text = DEFAULT_QUERY_PROMPT_TMPL.substitute(
        context_str=relevant_text,
        query_str=query
    )
    
    with console.status(f"[bold yellow]quering the model..."):
        model_out = query_model(prompt=query_text)
    
    console.log(f"[bold yellow]Question: {query}")
    console.log(f"[bold yellow]Context: {local_file.name}")
    console.log(f"[bold green]Answer: {model_out}")

In [24]:
query_pdf(
	pdf_url=PDF_URL,
	query="explain the attention mechanism"
)


[05:38:49] embedding index loaded from cache                                                         ]8;id=364261;file:///tmp/ipykernel_1705/41403016.py\41403016.py]8;;\:]8;id=200628;file:///tmp/ipykernel_1705/41403016.py#23\23]8;;\

           found relevant text from input                                                            ]8;id=286012;file:///tmp/ipykernel_1705/41403016.py\41403016.py]8;;\:]8;id=342464;file:///tmp/ipykernel_1705/41403016.py#83\83]8;;\

Output()

[05:40:17] Question: explain the attention mechanism                                                 ]8;id=848656;file:///tmp/ipykernel_1705/41403016.py\41403016.py]8;;\:]8;id=608898;file:///tmp/ipykernel_1705/41403016.py#93\93]8;;\

           Context: 1706.03762.pdf                                                                   ]8;id=10096;file:///tmp/ipykernel_1705/41403016.py\41403016.py]8;;\:]8;id=109292;file:///tmp/ipykernel_1705/41403016.py#94\94]8;;\

           Answer: The attention mechanism is a technique used in deep learning models, particularly ]8;id=875547;file:///tmp/ipykernel_1705/41403016.py\41403016.py]8;;\:]8;id=216781;file:///tmp/ipykernel_1705/41403016.py#95\95]8;;\
           in sequence transduction models like neural machine translation, to selectively focus on                
           certain parts of the input sequence. It is a weighting function that determines the                     
           importance of each input element in the output.                                                         
                                                                                                                   
           In the context of the Transformer model mentioned in the paper, the attention mechanism                 
           is a key component of the model's architecture. It is based on the idea of                              
           self-attention, which allows the model to learn relationships between different parts of                
           the input sequence without the need for complex recurrent or convolutional neural                       
           networks.                                                                                               
                                                                                                                   
           The attention mechanism works by first representing the input sequence as a matrix of                   
           query (Q), key (K), and value (V) vectors. These vectors are typically learned during the               
           training process. Then, a weighted sum of the value vectors is calculated based on the                  
           importance of each input element, as determined by the attention mechanism.                             
                                                                                                                   
           The attention mechanism is calculated as a scaled dot-product of the Q and K vectors,                   
           followed by a non-linear activation function. The scaling factor helps to prevent the                   
           attention mechanism from becoming unstable due to the large number of input elements.                   
                                                                                                                   
           In the Transformer model, the attention mechanism is used in three different ways:                      
                                                                                                                   
           1. Self-attention: The model learns to attend to different parts of the input sequence to               
           determine the importance of each element in the output.                                                 
           2. Encoder-decoder attention: The model learns to attend to the output sequence (encoder)               
           based on the input sequence (decoder).                                                                  
           3. Multi-head attention: The model learns multiple attention mechanisms in parallel, each               
           focusing on different aspects of the input sequence.                                                    
                                                                                                                   
           The attention mechanism allows the Transformer model to learn long-range dependencies and               
           complex relationships between input elements, leading to improved performance in tasks                  
           such as machine translation.<dummy32000>                                                                

In [29]:
from PIL import Image
import pytesseract

In [30]:
# Load your image
image_path = '/home/ubuntu/Site_Plan_New_10-2023_page-0001.jpg'
image = Image.open(image_path)

# Use pytesseract to do OCR on the image
text = pytesseract.image_to_string(image)
trimmed_text = text.strip()

print(trimmed_text)

ee

   

PLOT ayill

yl bbao
Land Site Plan

Gb i tlg GLa Lull gil
DEPARTMENT OF MUNICIPALITIES
AND TRANSPORT

3

  

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

  

 

 

 

 

Notes and Conditions:

» This plan is to clarify the boundaries and parameters of the property only. It
shall not be considered asa ttle deed, nor be used in place of it. The mentioned
iamensions and area are considered preliminary, and the actual area will be fixed
after surveying and according to nature

* Any alteration or modification of, or addition to the document
rendersit void

* This contract has been issued electronically, and its validity can be
verified through the link: https://smarthub. adm.gov.ae
2023/627086

Application Number. ‘alolaall aay

 

 

 

 

 

 

 

 

 

see Une py Y&Kla as py ¥ dash Jl lang gn epangl ala fa *
Anglo pole isu lal in ghj yall AO yy

i ja crac a hig ect users

-ulyll Jus 40 aia Go gaaill j4aig uigyiSll aidusall Ihe ya *

 

Municipality ‘A

In [34]:
def query_model(prompt):
	response = client.chat.completions.create(
		model="mistral",
		messages=[{"role": "user", "content": prompt}],
		temperature=0.01,
		max_tokens=512,
		top_p=1,
		frequency_penalty=0,
		presence_penalty=0
	)
	
	return response.choices[0].message.content

In [35]:
question = "please return values of  plot area and land no in a json format only consider meaningful words in english present"

In [36]:
#relevant_text = "Plot Area 510.00 m*/5,489.53 Ft? aul aun, Land No 540 Aull Plot Address 135-0-000-540 Saquiall olgie"
  
query_text = DEFAULT_QUERY_PROMPT_TMPL.substitute(
  context_str=trimmed_text,
  query_str=question
)



In [37]:
query_text

'Context information is below.\n\n----------\nee\n\n   \n\nPLOT ayill\n\nyl bbao\nLand Site Plan\n\nGb i tlg GLa Lull gil\nDEPARTMENT OF MUNICIPALITIES\nAND TRANSPORT\n\n3\n\n  \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n  \n\n \n\n \n\n \n\n \n\nNotes and Conditions:\n\n» This plan is to clarify the boundaries and parameters of the property only. It\nshall not be considered asa ttle deed, nor be used in place of it. The mentioned\niamensions and area are considered preliminary, and the actual area will be fixed\nafter surveying and according to nature\n\n* Any alteration or modification of, or addition to the document\nrendersit void\n\n* This contract has been issued electronically, and its validity can be\nverified through the link: https://smarthub. adm.gov.ae\n2023/627086\n\nApplication Number. ‘alolaall aay\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\nsee Une py Y&Kla as py ¥ dash Jl lang gn epangl

In [ ]:
"

In [40]:
%timeit query_model(query_text)

55.9 s ± 49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
